In [1]:
from src.rag.components.generator import LLamaCppGeneratorComponent

/Users/esp.py/Projects/Personal/end-to-end-rag/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
prompt = "You are a french news reporter"
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [3]:
llama_cpp_generator = LLamaCppGeneratorComponent(
    api_url="http://127.0.0.1:8001",
    model_name=model_name,
    prompt=prompt
)

In [4]:
llama_cpp_generator._ping_api()

True

In [5]:
summarization_prompt = """
Give a summary in french of the following document:
{{content}}
 Describes it in a style of a french new paper reporters.

Don't summarize each document separately, the content in all the documents should be summarized.

The summary should be in french not in English
"""

### Download the File

In [6]:
from src.shared.cloud_storage import BackBlazeCloudStorage

In [7]:
cloud_storage = BackBlazeCloudStorage(environment="prod")

In [8]:
from datetime import datetime

In [9]:
today = datetime.now().strftime("%Y-%m-%d")

In [10]:
today

'2024-10-31'

In [11]:
bucket_name = "congonews-clusters"
file_name = f"news-clusters-{today}.csv"

In [12]:
import pandas as pd

In [13]:
from tempfile import NamedTemporaryFile

In [14]:
today_news_file = cloud_storage.download_by_name(
    bucket_name=bucket_name, file_name=file_name)
with NamedTemporaryFile(delete=True, suffix=".csv") as temp_file:
    today_news_file.save_to(temp_file.name)
    today_news_df = pd.read_csv(temp_file.name, index_col=0)

In [15]:
def select_top_clusters(news_df: pd.DataFrame) -> pd.DataFrame:
    """ select the clusters with the more than two documents """
    cluster_counts = news_df["labels"].value_counts()
    labels_with_more_than_one = cluster_counts[cluster_counts > 1].index
    important_news_df = news_df.loc[news_df.labels.isin(
        labels_with_more_than_one)]
    return important_news_df

In [16]:
today_news_df = select_top_clusters(today_news_df)

In [17]:
today_news_df = today_news_df.sort_values(by="labels")

### Prompt

In [18]:
summaries = []
for id, news_group in today_news_df.groupby("labels"):
    news = news_group.content.str.cat(sep="\n")
    summary = llama_cpp_generator.run(
        template_values={"content": news}, prompt_template=summarization_prompt)
    summaries.append(summary)
    print(summary)
    print("---" * 10)

La République Démocratique du Congo a présenté sa loi de finances pour l'exercice 2025, présentée par la Première Ministre Judith Suminwa Tuluka. Ce projet ambitieux met l'accent sur le développement et l'investissement pour renforcer l'économie et améliorer le quotidien des Congolais. 

1. **Accroissement des crédits d'investissement** : Les crédits alloués aux investissements connaissent une hausse de 18,2% par rapport à 2024. Cette augmentation permet de faire passer leur part dans le budget général de 15,1% en 2024 à 48,4% en 2025, témoignant d'une priorité accordée aux infrastructures et au développement économique.

2. **Renforcement du secteur de la sécurité** : Le budget alloué au secteur de la sécurité augmente de 25,2%, visant à renforcer les forces armées et de sécurité. Cet investissement est essentiel pour garantir la paix et la stabilité dans le pays.

3. **Soutien à l'agriculture, à la pêche et à l'élevage** : Le secteur de l'agriculture, de la pêche et de l'élevage béné

In [19]:
today_news_df.labels.nunique()

17

In [20]:
today_news_df

,database_id,content,title,posted_at,url,labels
22,26037051,"octobre 31, 2024 RDC-Budget 2025: Les 14 point...",RDC-Budget 2025: Les 14 points clés de Judith ...,2024-10-31 00:00:00,https://depeche.cd/2024/10/31/rdc-budget-2025-...,3
3,25965917,La loi de Finances exercice 2025 présentée jeu...,Assemblée nationale : voici les 14 points fort...,2024-10-31 00:00:00,https://www.mediacongo.net/article-actualite-1...,3
15,25961846,Le budget alloué aux investissements connaît u...,Loi de Finances 2025 : les crédits alloués au ...,2024-10-31 12:34:02,https://actu30.cd/2024/10/loi-de-finances-2025...,3
13,25961798,Judith Suminwa Tuluka a présenté ce jeudi à l'...,"Loi de Finances 2025: Hausse de 18,2% des créd...",2024-10-31 17:41:00,https://actualite.cd/2024/10/31/loi-de-finance...,3
30,25965990,Un protocole d’accord portant création de la c...,RDC-Soudan du sud : signature d’un protocole d...,2024-10-31 00:00:00,https://www.mediacongo.net/article-actualite-1...,4
57,25976666,Après avoir signé le protocole d’accord porta...,Juba : Le VPM Jacquemin Shabani reçu par le pr...,2024-10-31 07:50:50,https://scooprdc.net/2024/10/31/juba-le-vpm-ja...,4
19,25961861,Au terme des échanges qui ont duré trois heure...,Tshisekedi après sa rencontre avec Museveni : ...,2024-10-31 05:08:30,https://actu30.cd/2024/10/tshisekedi-apres-sa-...,5
47,25966335,"Le président ougandais Yoweri Kaguta Museveni,...",RDC - Ouganda : Yoweri Museveni déterminé à co...,2024-10-31 00:00:00,https://www.mediacongo.net/article-actualite-1...,5
41,25966246,Le Chef de l’État congolais prend part ce jeud...,"Bujumbura : après le COMESA 2024, Félix Tshise...",2024-10-31 00:00:00,https://www.mediacongo.net/article-actualite-1...,5
48,25966347,Le Burundi partage une frontière de 236 kilomè...,Le Burundi vante ses relations de « bon voisin...,2024-10-31 00:00:00,https://www.mediacongo.net/article-actualite-1...,5
